In [1]:
from TwitterAPI import TwitterAPI
from TwitterAPI import TwitterPager
import json
import pandas as pd
import time

###FULL ARCHIVE KEYS
# api = TwitterAPI(consumer_key = 'WBi5cw4Fahq8nhorYcIwJ8TOy',
#                 consumer_secret = 'ezBNO3v3TmJxm2ESvzrZPEfht6aFtEUHS6S9i8pDhJ159t5odK',
#                 access_token_key = '1582827440-ShN1QU8qg2ZllL8TUv64edfLGtNpT8SsCBdFtPb',
#                 access_token_secret = '6ceCQAmdyNHdTBVBXQsBKEzSFpX5MyHKQVzt9SU6yfLvg')

##30 DAY KEYS
api = TwitterAPI(consumer_key = 'r1NRvjJgmsFlCashfOcRzaUAA',
                 consumer_secret = '5LvQfwrnsCdwzScxcffKujq4R41gJQwO0DpifcqND7M6OPG12Y',
                 access_token_key = '1582827440-gaQnkTMyGolC5l9VY5ieFeh8Btqg85dxLdC1VSY',
                 access_token_secret = 'FfpwmcXfPDjZlsXasvSKwnp6MSFGNzJ6zQdEq69376RWR')

##DEV ENVIRONMENT
PRODUCT = '30day'
#PRODUCT = 'fullarchive'
LABEL = 'Research'


In [10]:
##BOUNDING BOXES: from https://boundingbox.klokantech.com/ format:csv
##NYC: bounding_box:[-74.135353 40.635674 -73.801643 40.893625]
##Canalside: bounding_box:[-78.8797905618,42.8756162086,-78.8774945908,42.8791227573]


##TWEET ITERATOR
r = TwitterPager(api, 'tweets/search/%s/:%s' % (PRODUCT, LABEL),        
               {'query':'bounding_box:[-78.8797905618 42.8756162086 -78.8774945908 42.8791227573]',
                'maxResults':100
               } )

list_tweets = []
count = 1
for item in r.get_iterator(wait=10):
    if 'text' in item:
        coordinates = item['coordinates'] 
        tweet_date = item['created_at']
        tweet_text = item['text']
        tweet_user = item['user']
        if 'extended_tweet' in item.keys():
            tweet_text = item['extended_tweet']['full_text']
        tweet_row = {'long_lat':coordinates,
                    'date_utc':tweet_date,
                    'full_text':tweet_text,
                    'user':tweet_user['screen_name'],
                    'location':tweet_user['location'],
                    'geoenabled':tweet_user['geo_enabled']}
        list_tweets.append(tweet_row)
        count = count + 1
        print(count,item['text'])
    elif 'message' in item and item['code'] == 88:
        print('SUSPEND, RATE LIMIT EXCEEDED: %s\n' % item['message'])
        break

2 #custommade #countdowntoNas #lookformeatCanalside 8.30.19 @ Canalside Live Concert Series https://t.co/pljP71AzvT
3 Long weekend vibes 👗☀️😊👻❤️ @ Buffalo Waterfront https://t.co/btqG7TNCRw
4 @sociald1 @mikeness @ Canalside Live Concert Series https://t.co/BfTHLNgKn7
5 @floggingmolly @ Canalside Live Concert Series https://t.co/g9U3gdtSFZ
6 @floggingmolly @ Canalside Live Concert Series https://t.co/6VZ8wt0VRY
7 #TheDevilMakesThree @thedevilmakes3 djheppel68 @ Canalside Live Concert Series https://t.co/DM78t5uSC9
8 @buffalolibrary Library on Wheels is at EPIC story time at Canalside! https://t.co/6vLfZVmosG
9 Fun in the rain buffalowaterfront 
@floggingmolly. #floggingmolly #justsayingbuffalo 😁🌧😁 @ Buffalo Waterfront https://t.co/YN3TcVWrVl
10 HAPPINESS BEGINS https://t.co/k5eCtJCsxf
11 MAKE AMERICA GREAT AGAIN. I wish I had recorded the awkward silence as he let the crowd stew on the point he was ma… https://t.co/3DOqeIFnaS
12 Great road trip to see Social D! @sociald1  Good time had 

In [6]:
df = pd.DataFrame(list_tweets)
print(df)

                          date_utc  \
0   Thu Aug 29 23:30:07 +0000 2019   
1   Thu Aug 29 10:41:21 +0000 2019   
2   Wed Aug 28 20:39:21 +0000 2019   
3   Wed Aug 28 20:37:23 +0000 2019   
4   Wed Aug 28 20:36:26 +0000 2019   
5   Wed Aug 28 20:32:28 +0000 2019   
6   Wed Aug 28 15:00:03 +0000 2019   
7   Wed Aug 28 10:42:04 +0000 2019   
8   Wed Aug 28 06:14:42 +0000 2019   
9   Wed Aug 28 05:29:31 +0000 2019   
10  Wed Aug 28 04:18:36 +0000 2019   
11  Wed Aug 28 02:54:18 +0000 2019   
12  Wed Aug 28 02:03:29 +0000 2019   
13  Wed Aug 28 00:33:30 +0000 2019   
14  Wed Aug 28 00:14:03 +0000 2019   
15  Wed Aug 28 00:03:54 +0000 2019   
16  Tue Aug 27 22:43:28 +0000 2019   
17  Tue Aug 27 22:35:01 +0000 2019   
18  Tue Aug 27 16:51:53 +0000 2019   
19  Tue Aug 27 12:38:12 +0000 2019   
20  Mon Aug 26 01:42:25 +0000 2019   
21  Sun Aug 25 22:35:11 +0000 2019   
22  Sun Aug 25 21:04:59 +0000 2019   
23  Fri Aug 23 21:21:07 +0000 2019   
24  Thu Aug 22 14:57:45 +0000 2019   
25  Sun Aug 

In [ ]:
df.to_json('190830_30Day_Test.json', orient='records')
df.to_csv('190830_30Day_Test.csv')

In [ ]:
## DEPRECIATED; DO NOT RUN

list_tweets = []
next = None
web_request_count = 0

r = api.request('tweets/search/%s/:%s' % (PRODUCT, LABEL),        
               {'query':'bounding_box:[-74.135353 40.635674 -73.801643 40.893625]',
                'next': next # This has to be commented out for the first web request
               })

print('r.status_code: ', r.status_code)
print('web_request_count: ', web_request_count)
web_request_count += 1
results = r.json()['results']
for tweet in results:
    coordinates = tweet['coordinates'] 
    tweet_date = tweet['created_at']
    tweet_text = tweet['text']
    tweet_user = tweet['user']
    if 'extended_tweet' in tweet.keys():
        tweet_text = tweet['extended_tweet']['full_text']
    tweet_row = {'long_lat':coordinates,
                'date_utc':tweet_date,
                'full_text':tweet_text,
                'user':tweet_user['screen_name'],
                'location':tweet_user['location'],
                'geoenabled':tweet_user['geo_enabled']}
    list_tweets.append(tweet_row)
df = pd.DataFrame(list_tweets)
print(df)

In [ ]:
## DEPRECIATED; DO NOT RUN
df.to_json('Test5_Loc_20180829.json', orient='records')
df.to_csv('Test5_Loc_20180829.csv')

In [ ]:
## DEPRECIATED; DO NOT RUN

while next is not None:
       r = api.request('tweets/search/%s/:%s' % (PRODUCT, LABEL),        
               {'query':'bounding_box:[-78.8797476464 42.8766147304 -78.8771083527 42.8791735224]',
                'toDate':'201812310000',
                'fromDate':'201801010000',
                'maxResults':'500',
                'next': next
                })
       print('r.status_code: ', r.status_code)
       next = r.json()['next']
       print('web_request_count: ', web_request_count)
       web_request_count += 1
       results = r.json()['results']
       for tweet in results:
           coordinates = tweet['coordinates']['coordinates']
           tweet_date = tweet['created_at']
           tweet_text = tweet['text']
           tweet_user = tweet['user']
           if 'extended_tweet' in tweet.keys():
               tweet_text = tweet['extended_tweet']['full_text']
           tweet_row = {'long_lat':coordinates,
                        'date_utc':tweet_date,
                        'full_text':tweet_text,
                        'user':tweer_user}
           list_tweets.append(tweet_row)
       df = pd.DataFrame(list_tweets)
           
       df.to_json('tweets_df_buffalo_25mi_18.json', orient='records')
       df.to_csv('tweets_df_buffalo_25mi_18.csv', orient='records')
       time.sleep(2.1)

In [ ]:
print(df)